In [2]:
%pip install valentine

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pandas

  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl (10.3 MB)
  Using cached numpy-1.24.2-cp311-cp311-win_amd64.whl (14.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] Impossibile trovare il file specificato: 'c:\\Python311\\Scripts\\f2py.exe' -> 'c:\\Python311\\Scripts\\f2py.exe.deleteme'



# CONTROLLO MATCHING

In [3]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased, Cupid
import pprint

# Load data using pandas
df1 = pd.read_json("schema_mediato_confronto.json")
# df2 = pd.read_json("datasets/output_globaldata.json")
df2 = pd.read_json("datasets/valueToday2.json")


# Instantiate matcher and run
# Coma requires java to be installed on your machine
# If java is not an option, all the other algorithms are in Python (e.g., Cupid)
# matcher = Coma(strategy="COMA_OPT")
matcher = DistributionBased(0.9, 0.9)
matches = valentine_match(df1, df2, matcher)
# print(type(matches))
# print(matches)
pp = pprint.PrettyPrinter(indent=4)

In [4]:
pp.pprint(matches)

{   (('table_1', 'country'), ('table_2', 'country')): 0.9068413547766185,
    (('table_1', 'employees'), ('table_2', 'world_rank')): 0.7912659014719268,
    (('table_1', 'name'), ('table_2', 'name')): 0.889011487517116}


# CREAZIONE SCHEMA MEDIATO

In [19]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased
import pprint

In [20]:
# ---------------------------------------------------------
# CREATA PER INIZIALIZZARE UN DATASET FITTIZIO MA ORA NON
# SI UTILIZZA IL DATASET FITTIZIO DA NOI CREATO MANUALMENTE
# ---------------------------------------------------------
def creation_dataset_confronto(df1):
  # Load data using pandas
  df2 = pd.read_json("datasets/output_disfold.json")

  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])

  for i in df2.index:
      data = {}
      # print(i)
      for d,m in zip(attributi_dataset,attributi_mediato):
        # print(df2[d][i])
        data[m] = df2[d][i]
      temp_data=pd.DataFrame([data])
      df1=pd.concat([df1, temp_data], ignore_index=True)

  return df1


In [21]:
def creation_df_log_matcher(df_logmachers, matches, i, type):
  record = {}
  record['tab1'] = i[0][0]
  record['attr1'] = i[0][1]
  record['tab2'] = i[1][0]
  record['attr2'] = i[1][1]
  record['prob'] = matches[i]
  if type == 'coma':
    record['matcher'] = 'Coma'
  else:
    record['matcher'] = 'DistributionBased'
  temp_record=pd.DataFrame([record])
  # df_logmachers=df_logmachers.append(record, ignore_index=True)
  df_logmachers=pd.concat([df_logmachers, temp_record], ignore_index=True)
  return df_logmachers

In [22]:
def creation_schema_alignment(df1, df2, df_confronto, df_logmachers):

  # Instantiate matcher and run
  # Coma requires java to be installed on your machine
  # If java is not an option, all the other algorithms are in Python (e.g., Cupid)
  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  distr_run = False

 
  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    df_logmachers = creation_df_log_matcher(df_logmachers, matches_coma, i, 'coma')
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])
    else:
      if not distr_run:
        matcher_distr = DistributionBased(0.9, 0.9)
        matches_distr = valentine_match(df_confronto, df2, matcher_distr)
        distr_run = True
      try:
        df_logmachers = creation_df_log_matcher(df_logmachers, matches_distr, i, 'distribution')
        if matches_distr[i] > 0.5:
          attributi_mediato.append(i[0][1])
          attributi_dataset.append(i[1][1])
      except:
        pass
  # print(attributi_dataset)
  # print(attributi_mediato)

  # inserimento dei dati nel dataframe
  for i in df2.index:
    data = {}
    # print(i)
    for d,m in zip(attributi_dataset,attributi_mediato):
      # print(df2[d][i])
      data[m] = df2[d][i]
    temp_data=pd.DataFrame([data])
    df1=pd.concat([df1, temp_data], ignore_index=True)

  pp = pprint.PrettyPrinter(indent=4)

  # stampa dei matchers
  pp.pprint(matches_coma)
  if distr_run:
    pp.pprint(matches_distr)

  return df1, df_logmachers
  # print(df1)


In [25]:
from pathlib import Path
#formati json, csv, xls, xlsx, jsonl
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'share_price', 'website', 'stock', 'area_served', 'earnings'])
# df_confronto = creation_dataset_confronto(df1)
df_confronto = pd.read_json('schema_mediato_confronto.json')
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])

#numero di errori nell'esecuzione di file
num_error = 0

directory = 'datasets'
dataset_list_json = Path(directory).glob('*.json')
dataset_list_jsonl = Path(directory).glob('*.jsonl')
dataset_list_csv = Path(directory).glob('*.csv')
dataset_list_xls = Path(directory).glob('*.xls')
dataset_list_xlsx = Path(directory).glob('*.xlsx')
i = 0

# --------------------------------------------------------------------
# Modificare seleziona_tipo per scegliere che tipo di file eseguire
# --------------------------------------------------------------------
seleziona_tipo = 2

# CON SELEZIONA_TIPO=0 SI RUNNANO I TUTTI TIPI DI FILE
# CON SELEZIONA_TIPO=1 SI RUNNANO I JSON
if seleziona_tipo==0 or seleziona_tipo==1:
  for path in dataset_list_json:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=2 SI RUNNANO I JSONL
if seleziona_tipo==0 or seleziona_tipo==2:
  for path in dataset_list_jsonl:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path, lines=True)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=3 SI RUNNANO I CSV
if seleziona_tipo==0 or seleziona_tipo==3:
  for path in dataset_list_csv:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_csv(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=4 SI RUNNANO I XLS
if seleziona_tipo==0 or seleziona_tipo==4:
  for path in dataset_list_xls:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=5 SI RUNNANO I XLSX
if seleziona_tipo==0 or seleziona_tipo==5:
  for path in dataset_list_xlsx:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


print("Numero errori: " + str(num_error))
display(df1)
print('--------DATAFRAME LOG MATCHERS--------')
display(df_logmachers)

0 datasets\companiesMarketCap_dataset.jsonl
ERRORE datasets\companiesMarketCap_dataset.jsonl
1 datasets\disfold_dataset.jsonl
{   (('table_1', 'ceo'), ('table_2', 'ceo')): 0.8310757,
    (('table_1', 'employees'), ('table_2', 'employees')): 0.84929436,
    (('table_1', 'founded'), ('table_2', 'founded')): 0.8446504,
    (('table_1', 'headquarters'), ('table_2', 'headquarters_country')): 0.6355629,
    (('table_1', 'market_cap'), ('table_2', 'market_cap')): 0.851268,
    (('table_1', 'name'), ('table_2', 'name')): 0.83519655}
2 datasets\hitHorizons_dataset.jsonl
{   (('table_1', 'address'), ('table_2', 'address')): 0.797828,
    (('table_1', 'earnings'), ('table_2', 'sic_code')): 0.275279,
    (('table_1', 'founded'), ('table_2', 'id')): 0.29596865,
    (('table_1', 'industry'), ('table_2', 'industry')): 0.80027896,
    (('table_1', 'name'), ('table_2', 'name')): 0.7883742}
{}
3 datasets\valueToday_dataset.jsonl
ERRORE datasets\valueToday_dataset.jsonl
Numero errori: 2


,name,industry,market_cap,employees,country,headquarters,address,sector,ceo,founded,share_price,website,stock,area_served,earnings
0,Apple,NaN,$2.825 Trillion,"100,000",NaN,Cupertino,NaN,NaN,Mr. Timothy D. Cook,"April 1, 1976",NaN,NaN,NaN,NaN,NaN
1,Danaher,NaN,$208.03 Billion,"78,000",NaN,Washington,NaN,NaN,Mr. Rainer M. Blair,1969,NaN,NaN,NaN,NaN,NaN
2,Merck,NaN,$210.38 Billion,"67,000",NaN,Kenilworth,NaN,NaN,Mr. Robert M. Davis J.D.,1891,NaN,NaN,NaN,NaN,NaN
3,Bhp,NaN,A$281.66 Billion,"40,110",NaN,Melbourne,NaN,NaN,"Mr. Mike P. Henry B.Sc., BSc (Chem)",1885,NaN,NaN,NaN,NaN,NaN
4,Nike,NaN,$213.60 Billion,"73,300",NaN,Beaverton,NaN,NaN,Mr. John J. Donahoe II,"January 25, 1964",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,Legacy Srl,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIA DELLA ZECCA 2, BOLOGNA, 40121, BOLOGNA, ITALY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1663,Cribis Holding Srl,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIA DELLA ZECCA 2, BOLOGNA, 40121, BOLOGNA, ITALY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1664,Porsche Italia Spa,Retail Trade,NaN,NaN,NaN,NaN,"CORSO STATI UNITI 35, PADOVA, 35127, PADOVA, I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1665,Intesa Sanpaolo Rbm Salute Spa,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIALE STELVIO 55/57, MILANO, 20159, MILANO, ITALY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


--------DATAFRAME LOG MATCHERS--------


,tab1,attr1,tab2,attr2,prob,matcher
0,table_1,market_cap,table_2,market_cap,0.851268,Coma
1,table_1,employees,table_2,employees,0.849294,Coma
2,table_1,founded,table_2,founded,0.844650,Coma
3,table_1,name,table_2,name,0.835197,Coma
4,table_1,ceo,table_2,ceo,0.831076,Coma
5,table_1,headquarters,table_2,headquarters_country,0.635563,Coma
6,table_1,industry,table_2,industry,0.800279,Coma
7,table_1,address,table_2,address,0.797828,Coma
8,table_1,name,table_2,name,0.788374,Coma
9,table_1,founded,table_2,id,0.295969,Coma


In [51]:
df1.to_csv('schema_mediato.csv')

In [52]:
df_logmachers.to_csv('log_matchers.csv')

# Il codice sottostante è utilizzato per generare il dataframe di un singolo dataset

In [37]:
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'share_price', 'website', 'stock', 'area_served', 'revenue'])
df_confronto = pd.read_json('schema_mediato_confronto.json')
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])


# df2 = pd.read_json('datasets/output_globaldata.json')
df2 = pd.read_json('datasets/valueToday2.json')
df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)

df1
df_logmachers

{   (('table_1', 'ceo'), ('table_2', 'CEO')): 0.82983994,
    (('table_1', 'country'), ('table_2', 'country')): 0.8339778,
    (('table_1', 'market_cap'), ('table_2', 'marketCap')): 0.7074935,
    (('table_1', 'name'), ('table_2', 'name')): 0.8309938,
    (('table_1', 'stock'), ('table_2', 'world_rank')): 0.32348847}
{(('table_1', 'country'), ('table_2', 'country')): 0.9626358793204958}


,tab1,attr1,tab2,attr2,prob,matcher
0,table_1,country,table_2,country,0.833978,Coma
1,table_1,name,table_2,name,0.830994,Coma
2,table_1,ceo,table_2,CEO,0.829840,Coma
3,table_1,market_cap,table_2,marketCap,0.707493,Coma
4,table_1,stock,table_2,world_rank,0.323488,Coma
